In [7]:
import pandas as pd
kinetics = pd.read_csv('./data/overall_distances_kinetics.csv', index_col=0)
kinetics = kinetics.round(0)
kinetics

,0,1,2,3,4,5,6,7,8,9
0,0.0,369.0,850.0,400.0,960.0,158.0,2408.0,385.0,326.0,5342.0
1,369.0,0.0,615.0,521.0,890.0,441.0,2482.0,689.0,294.0,5385.0
2,850.0,615.0,0.0,927.0,970.0,884.0,2578.0,1031.0,822.0,5435.0
3,400.0,521.0,927.0,0.0,1026.0,493.0,2072.0,642.0,407.0,5031.0
4,960.0,890.0,970.0,1026.0,0.0,991.0,2575.0,1121.0,933.0,5433.0
5,158.0,441.0,884.0,493.0,991.0,0.0,2454.0,280.0,413.0,5377.0
6,2408.0,2482.0,2578.0,2072.0,2575.0,2454.0,0.0,2447.0,2345.0,3047.0
7,385.0,689.0,1031.0,642.0,1121.0,280.0,2447.0,0.0,665.0,5370.0
8,326.0,294.0,822.0,407.0,933.0,413.0,2345.0,665.0,0.0,5229.0
9,5342.0,5385.0,5435.0,5031.0,5433.0,5377.0,3047.0,5370.0,5229.0,0.0


In [4]:
init = pd.read_csv('./data/initial_condition.csv', index_col=0)
init

,leiden
0,85
1,75
2,68
3,65
4,60
5,55
6,48
7,45
8,31
9,18


In [22]:
import numpy as np
factor = pd.read_csv('./data/kinetics_array_correction_factor.txt', sep=' ', header=None).astype(np.int32)
factor.to_csv('./data/kinetics_array_mz.txt', index=None, header=None, sep=' ')

In [2]:
anno = pd.read_csv('./data/annotations.csv')
anno

,clones,populations
0,0,HSC_MPP
1,1,MEMP
2,2,Megakaryocyte
3,3,Mast cell
4,4,Early Erythroid
5,5,Mid Erythroid
6,6,Late Erythroid
7,7,Neutrophil-myeloid progenitor
8,8,DC
9,9,Monocyte precursor
